This notebook exports data from the tensorboard for several metrics/losses simultaneously.
The code is based on:
https://stackoverflow.com/questions/72837772/download-all-of-csv-files-of-tensorboard-at-once

In [1]:
import pandas as pd
import numpy as np
import requests
from csv import reader

In [2]:
def tb_data(version, split):

    # Get epoch data
    url = f'http://localhost:6006/experiment/defaultExperimentId/data/plugin/scalars/scalars?tag=epoch&run=version_{version}&format=csv'
    r = requests.get(url, allow_redirects=True)
    data_csv = list(reader(r.text.splitlines()))
    result_df = pd.DataFrame(data_csv[1:], columns=data_csv[0])
    result_df.drop(columns=['Wall time'], inplace=True)
    result_df.rename(columns={'Step': 'step', 'Value': 'epoch'}, inplace=True)
    result_df['epoch'] = result_df['epoch'].astype(float)
    result_df['step'] = result_df['step'].astype(float)
    # Fill in missing steps (carry the last known epoch)
    full_steps = np.arange(1, result_df["step"].max() + 1)
    result_df = (
        pd.merge_asof(                     # fast & memory‑friendly
            pd.DataFrame({"step": full_steps}),
            result_df.sort_values("step"),
            on="step",
            direction="forward"           # take the most recent epoch ≤ step
        )
    )

    # Get other data
    dfs = {}
    possible_metrics = ['disc_loss', 'gen_loss', 'kl_loss', 'nll_loss', 'perc_img_loss', 'rec_img_loss', 'rec_lat_loss', 
                        'total_loss', 'disc_active', 'fid_score', 'spectral_score']
    possible_intervals = ['', '_step', '_epoch']

    for metric in possible_metrics:
        for interval in possible_intervals:
            key = f'{split}/{metric}{interval}'
            url = f'http://localhost:6006/experiment/defaultExperimentId/data/plugin/scalars/scalars?tag={split}%2F{metric}{interval}&run=version_{version}&format=csv'
            
            try:
                r = requests.get(url, allow_redirects=True)
                data_csv = list(reader(r.text.splitlines()))
                df = pd.DataFrame(data_csv[1:], columns=data_csv[0])
                if not df.empty:
                    df.drop(columns=['Wall time'], inplace=True)
                    df.rename(columns={'Step': 'step', 'Value': key}, inplace=True)
                    df['step'] = df['step'].astype(float)
                    dfs[key] = df
                    break
            except:
                print(f"{key} not found.")
                continue

    # Merge dataframes
    for key, df in dfs.items():
        result_df = pd.merge(result_df, df, on='step', how='right')
    
    # Sort by 'Step' (int)
    result_df.sort_values(by='step', inplace=True)
    result_df.reset_index(drop=True, inplace=True)
    
    return result_df

In [3]:
MODEL = 'latent_vae'
VERSION = '20'
SPLIT = 'val'

df = tb_data(VERSION, SPLIT)

# export df to csv
df.to_csv(f'losses/{MODEL}_v{VERSION}_{SPLIT}_losses.csv', index=False)